<h1>Capstone Project - The Battle of Neighborhoods</h1>

<h3>Introduction</h3>

<p>Selecting the best location to open a mexican food restaurant in Manhattan, New York</p>


<p>The City of New York is famous for its excelllent cuisine. It's food culture includes an array of international cuisines influenced by the city's immigrant history.</p>

<p>Mexican food is very popular around the world, its delicious flavour and spicy taste so starting a mexican food restaurant can be a great business opportunity, but you need to distinguish yourself from others to enjoy long-term success.</p>

<h3>Bussiness Problem</h3>

<p>My client wants to open his business in Manhattan area, so I focus on that borough during my analysis. We define potential neighborhood based on the number of mexican food restaurant which are operating right in each neighborhood. Manhattan has full potential but also is a very challenging district to open a business because of high competition. New mexican food restaurant should be open in an area that inadequate neighborhood in this way the bar can attract more customers. Therefore, this analysis necessary to ensure that we have enough customers and that we are not so close to other restaurants.</p>

<h3>Data</h3>

<p>To identify the characteristics of our competitors' venues in Manhattan, we would first need to find out the number of mexican food restaurants in Manhattan currently and their location. We then used Google Map API to find their geographic coordinates based on their postal code addresses.
</p>